In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from torch.nn.parallel import DistributedDataParallel as DDP

# 환경 변수 설정
# os.environ["RANK"] = "0"  # 첫 번째 프로세스 (분산 학습에서 프로세스 순서)
# os.environ["WORLD_SIZE"] = "2"  # 총 GPU 수 (2개 GPU 사용)
# os.environ["LOCAL_RANK"] = "0"  # 첫 번째 GPU (0번 GPU)
# os.environ["MASTER_ADDR"] = "localhost"  # 마스터 노드의 주소
# os.environ["MASTER_PORT"] = "1234"  # 마스터 노드의 주소

c:\anaconda3\envs\deepseek\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# torch.set_num_threads(4) 
# # 여러 GPU 사용 시, 초기화
# torch.distributed.init_process_group(backend="nccl")


In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", #nf4
    bnb_4bit_use_double_quant=True, #True
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained("./deepseek-coder-7b-instruct-v1.5", trust_remote_code=True, quantization_config=bnb_config, torch_dtype=torch.float16 ).to('cuda')
#model = DDP(model)

tokenizer = AutoTokenizer.from_pretrained("./deepseek-coder-7b-instruct-v1.5", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.66s/it]


In [27]:
messages = [
    {
        'role': 'user',
        'content': """
        퀵소트 알고리즘을 작성해주세요
        1. Explain the algorithm in detail.
        2. Write the code and provide comments for each part of the code.
        3. Answer in Korean.
        """
    }
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to('cuda')

# attention_mask가 없는 경우 자동 생성 (기본적으로 1로 채워진 값으로 모든 토큰을 유효한 값으로 처리)
attention_mask = torch.ones(inputs.size(), device=inputs.device)
outputs = model.generate(inputs, 
                         attention_mask=attention_mask, 
                         max_new_tokens=1024, 
                         do_sample=False, 
                         top_k=50, 
                         top_p=0.95, 
                         num_return_sequences=1, 
                         eos_token_id=tokenizer.eos_token_id,
                         pad_token_id=tokenizer.pad_token_id
                         ).to('cuda')

print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))

c:\anaconda3\envs\deepseek\lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


퀵소트(Quick Sort)는 분할 정복 알고리즘의 일종으로, 리스트를 두 개의 서브리스트로 분할하고 각각을 정렬하는 방식으로 작동합니다. 알고리즘의 각 단계는 다음과 같습니다:

1. 리스트에서 한 요소를 선택합니다(이를 피벗이라고 함).
2. 피벗보다 작은 요소들은 피벗의 왼쪽으로, 큰 요소들은 오른쪽으로 이동시킵니다.
3. 피벗을 중심으로 피벗을 포함하지 않은 두 서브리스트에 대해 재귀적으로 퀵소트를 적용합니다.
4. 모든 서브리스트가 정렬될 때까지 이 과정을 반복합니다.

이제 퀵소트의 코드를 작성하겠습니다:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quick_sort(left) + middle + quick_sort(right)

# 테스트
print(quick_sort([3,6,8,10,1,2,1]))
# 출력: [1, 1, 2, 3, 6, 8, 10]
```

이 코드는 피벗을 리스트의 중간 요소로 선택하고, 피벗보다 작은 요소들은 왼쪽, 피벗보다 큰 요소들은 오른쪽으로 이동시킵니다. 그런 다음 피벗을 제외한 두 서브리스트에 대해 재귀적으로 퀵소트를 적용합니다. 이 과정은 리스트의 길이가 1 이하가 될 때까지 반복됩니다.

이 코드는 리스트의 길이가 1 이하일 때 정렬이 필요 없으므로, 이를 조건으로 사용하여 정렬이 필요 없는 경우를 최대한 피합니다.
